In [10]:
import pandas as pd
import numpy as np
from untrade.client import Client

In [11]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate 13-period Exponential Moving Average (EMA)
    df['EMA_13'] = df['close'].ewm(span=13, adjust=False).mean()
    
    # Calculate 14-period Relative Strength Index (RSI)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Calculate average volume over the last 26 candles
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(26, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_rsi = df['RSI'].iloc[i]
        curr_ema = df['EMA_13'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if curr_close > curr_ema and curr_rsi > 50 and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = 2
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.2 * delta)
                df.loc[df.index[i], 'TP'] = curr_close + (3 * delta)
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif curr_close < curr_ema and curr_rsi < 50 and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = -2
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_low + (0.2 * delta)
                df.loc[df.index[i], 'TP'] = curr_close - (3 * delta)
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP', "EMA_13"]]
    
    return df




def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type', "EMA_13"]]
    
    data["leverage"] = 5
    data["position"] = 100
    data["quantity"] = 1000
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        position=50,  # Set initial position sizeq
        quantity=1000,  # Set quantity for backtest
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 460,
      "Leverage Applied": 5.0,
      "Winning Trades": 205,
      "Losing Trades": 255,
      "No. of Long Trades": 233,
      "No. of Short Trades": 227,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 44.565217,
      "Winning Streak": 4,
      "Losing Streak": 10,
      "Gross Profit": 12376.492264,
      "Net Profit": 8926.492264,
      "Average Profit": 19.405418,
      "Maximum Drawdown(%)": 112.342706,
      "Average Drawdown(%)": 26.032694,
      "Largest Win": 3638.744131,
      "Average Win": 262.420135,
      "Largest Loss": -1787.664697,
      "Average Loss": -175.959355,
      "Maximum Holding Time": "20 days 15:59:59",
      "Average Holding Time": "2 days 10:24:31",
      "Maximum Adverse Excursion": 

In [12]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 460,
      "Leverage Applied": 5.0,
      "Winning Trades": 205,
      "Losing Trades": 255,
      "No. of Long Trades": 233,
      "No. of Short Trades": 227,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 44.565217,
      "Winning Streak": 4,
      "Losing Streak": 10,
      "Gross Profit": 12376.492264,
      "Net Profit": 8926.492264,
      "Average Profit": 19.405418,
      "Maximum Drawdown(%)": 112.342706,
      "Average Drawdown(%)": 26.032694,
      "Largest Win": 3638.744131,
      "Average Win": 262.420135,
      "Largest Loss": -1787.664697,
      "Average Loss": -175.959355,
      "Maximum Holding Time": "20 days 15:59:59",
      "Average Holding Time": "2 days 10:24:31",
      "Maximum Adverse Excursion": 

In [13]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_6h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 299,
      "Leverage Applied": 5.0,
      "Winning Trades": 146,
      "Losing Trades": 153,
      "No. of Long Trades": 151,
      "No. of Short Trades": 148,
      "Benchmark Return(%)": 1442.288422,
      "Benchmark Return(on $1000)": 14422.884225,
      "Win Rate": 48.829431,
      "Winning Streak": 5,
      "Losing Streak": 7,
      "Gross Profit": 16136.934768,
      "Net Profit": 13894.434768,
      "Average Profit": 46.469681,
      "Maximum Drawdown(%)": 37.996242,
      "Average Drawdown(%)": 7.77488,
      "Largest Win": 2722.802175,
      "Average Win": 306.569623,
      "Largest Loss": -654.681711,
      "Average Loss": -201.730263,
      "Maximum Holding Time": "42 days 18:0:0",
      "Average Holding Time": "3 days 20:41:20",
      "Maximum Adverse Excursion": 60.32

In [14]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_8h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 213,
      "Leverage Applied": 5.0,
      "Winning Trades": 106,
      "Losing Trades": 107,
      "No. of Long Trades": 106,
      "No. of Short Trades": 107,
      "Benchmark Return(%)": 1457.640166,
      "Benchmark Return(on $1000)": 14576.401662,
      "Win Rate": 49.765258,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 17505.592164,
      "Net Profit": 15908.092164,
      "Average Profit": 74.685879,
      "Maximum Drawdown(%)": 43.057729,
      "Average Drawdown(%)": 9.269467,
      "Largest Win": 4524.02532,
      "Average Win": 404.200522,
      "Largest Loss": -895.126889,
      "Average Loss": -251.749188,
      "Maximum Holding Time": "48 days 15:59:59",
      "Average Holding Time": "5 days 8:9:0",
      "Maximum Adverse Excursion": 64.067

In [15]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_12h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 132,
      "Leverage Applied": 5.0,
      "Winning Trades": 67,
      "Losing Trades": 65,
      "No. of Long Trades": 67,
      "No. of Short Trades": 65,
      "Benchmark Return(%)": 1467.065217,
      "Benchmark Return(on $1000)": 14670.652174,
      "Win Rate": 50.757576,
      "Winning Streak": 5,
      "Losing Streak": 8,
      "Gross Profit": 17365.982274,
      "Net Profit": 16375.982274,
      "Average Profit": 124.060472,
      "Maximum Drawdown(%)": 19.742881,
      "Average Drawdown(%)": 5.395042,
      "Largest Win": 4540.64158,
      "Average Win": 516.788975,
      "Largest Loss": -1876.306274,
      "Average Loss": -280.751985,
      "Maximum Holding Time": "63 days 12:0:0",
      "Average Holding Time": "8 days 23:43:38",
      "Maximum Adverse Excursion": 65.8025

In [16]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_1d.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_13960/970004699.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 56,
      "Leverage Applied": 5.0,
      "Winning Trades": 27,
      "Losing Trades": 29,
      "No. of Long Trades": 30,
      "No. of Short Trades": 26,
      "Benchmark Return(%)": 1462.97378,
      "Benchmark Return(on $1000)": 14629.737803,
      "Win Rate": 48.214286,
      "Winning Streak": 3,
      "Losing Streak": 5,
      "Gross Profit": 20972.95877,
      "Net Profit": 20552.95877,
      "Average Profit": 367.017121,
      "Maximum Drawdown(%)": 51.644892,
      "Average Drawdown(%)": 8.234321,
      "Largest Win": 15120.293492,
      "Average Win": 1265.657278,
      "Largest Loss": -1939.740284,
      "Average Loss": -469.647853,
      "Maximum Holding Time": "142 days 0:0:0",
      "Average Holding Time": "18 days 15:51:25",
      "Maximum Adverse Excursion": 61.8723

In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

def filter_signals(df):
    """Filter out consecutive duplicate trade signals while keeping only the first occurrence in a sequence."""
    df = df.copy()
    df['filtered_signals'] = df['signals'].where(df['signals'] != df['signals'].shift(), 0)
    return df

def visualize_signals_interactive(df: pd.DataFrame, window: int = 40):
    df = filter_signals(df)  # Apply signal filtering
    trade_indices = df[df['filtered_signals'] != 0].index.tolist()  # Get non-zero trade points
    
    if not trade_indices:
        print("No trade signals found!")
        return
    
    slider = widgets.IntSlider(min=0, max=len(trade_indices) - 1, step=1, description="Trade Index")
    output = widgets.Output()
    
    def update_plot(trade_idx):
        with output:
            clear_output()
            idx = trade_indices[trade_idx]
            start = max(idx - window, 0)
            end = min(idx + window, len(df) - 1)

            fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                                subplot_titles=("Candlestick Chart with EMA and Trade Signals",))
            
            # Candlestick Chart
            fig.add_trace(go.Candlestick(x=df['datetime'][start:end],
                                         open=df['open'][start:end],
                                         high=df['high'][start:end],
                                         low=df['low'][start:end],
                                         close=df['close'][start:end]),
                         row=1, col=1)
            
            # EMA Line
            fig.add_trace(go.Scatter(x=df['datetime'][start:end],
                                     y=df['EMA_13'][start:end],
                                     name="EMA 13",
                                     line=dict(color='blue', width=2)),
                         row=1, col=1)
            
            long_signals = df[df['filtered_signals'] == 1].loc[start:end]
            short_signals = df[df['filtered_signals'] == -1].loc[start:end]
            
            # Long Signal (Blue)
            fig.add_trace(go.Scatter(x=long_signals['datetime'], 
                                     y=long_signals['close'],
                                     mode='markers',
                                     marker=dict(color='blue', size=10, symbol='triangle-up'),
                                     name='Long Signal'),
                         row=1, col=1)
            
            # Short Signal (Black)
            fig.add_trace(go.Scatter(x=short_signals['datetime'], 
                                     y=short_signals['close'],
                                     mode='markers',
                                     marker=dict(color='black', size=10, symbol='triangle-down'),
                                     name='Short Signal'),
                         row=1, col=1)

            fig.update_layout(title=f"Trade Signal Visualization at Index {idx}",
                              xaxis_rangeslider_visible=False,
                              showlegend=True)
            fig.show()
    
    display(slider, output)
    
    widgets.interactive(update_plot, trade_idx=slider)
    update_plot(0)

In [18]:
visualize_signals_interactive(res)

IntSlider(value=0, description='Trade Index', max=151)

Output()